In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import traceback
import os
import pathlib
from keras_preprocessing.image import ImageDataGenerator
import datautils
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('./data/train_merge.csv')
val_df = pd.read_csv('./data/val_merge.csv')
train_dir = './data/train_set'
val_dir = './data/val_set'

In [9]:
sample = 50000
X_train, y_train = datautils.gettrain(train_df, train_dir, sample=sample, image_size=128, batch_size=1)

Found 50000 validated image filenames belonging to 251 classes.


In [10]:
np.save(f'X_train{sample}.npy', X_train)
np.save(f'y_train{sample}.npy', y_train)

In [ ]:
X_train, y_train = np.load('X_train6000.npy'), np.load('y_train6000.npy')

In [11]:
X_train.shape

(50000, 49152)

In [13]:
y_train.shape

(50000,)

In [15]:
X_train, y_train = np.load('X_train6000.npy'), np.load('y_train6000.npy')

In [16]:
lda = LinearDiscriminantAnalysis(store_covariance=True)
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=True, tol=0.0001)

## Evaluate

### Train accuracy

In [17]:
pred_train = lda.predict(X_train)
pred_train_acc = np.mean(pred_train == y_train)
print(f'Train accuracy is {pred_train_acc}')

Train accuracy is 0.74598


### Validation accuracy

In [19]:
X_val, y_val, label = datautils.gettestdata(val_df, val_dir, sample=len(val_df), class_mode='class')

Found 11994 validated image filenames belonging to 251 classes.


In [20]:
X_val.shape

(11994, 3072)

In [21]:
pred_val = lda.predict(X_val)
pred_val_acc = np.mean(pred_val == y_val)
print(f'Validation accuracy is {pred_val_acc}')

Validation accuracy is 0.009254627313656828


## Test

In [22]:
test_df = pd.read_csv('./data/sample_submission.csv')
test_df['label'] = test_df['label'].astype(str)
test_dir = './data/test_set'

In [23]:
X_test, _, _ = datautils.gettestdata(test_df, test_dir, sample=len(test_df), class_mode='label')

Found 28377 validated image filenames belonging to 1 classes.


In [24]:
X_test.shape

(28377, 3072)

In [25]:
pred_test = lda.predict(X_test)

In [26]:
pred_test

array([135, 131,  44, ..., 190,  33, 237])

In [27]:
n = len(pred_test)
pred_class = []
for i in range(n):
    pred_class.append(label[pred_test[i]])

In [28]:
test_df['class'] = pred_class

In [29]:
test_df

,img_name,label,class
0,test_024088.jpg,250,lobster_bisque
1,test_024089.jpg,250,knish
2,test_024090.jpg,250,chicken_provencale
3,test_024091.jpg,250,haggis
4,test_024092.jpg,250,toad_in_the_hole
...,...,...,...
28372,test_024083.jpg,250,osso_buco
28373,test_024084.jpg,250,vermicelli
28374,test_024085.jpg,250,roulade
28375,test_024086.jpg,250,cannoli


In [30]:
table = pd.read_table('./data/class_list.txt', header=None, sep=' ')
table.columns = ['label', 'class']

In [31]:
class_dir = {}
n = len(table)
for i in range(n):
    class_dir[table.iloc[i]['class']] = table.iloc[i]['label']

In [32]:
n = len(pred_test)
for i in range(n):
    test_df.loc[i, 'label'] = class_dir[test_df.iloc[i]['class']]

In [33]:
test_df.to_csv('submission.csv')

## QDA

In [2]:
X_train, y_train = np.load('X_train6000.npy'), np.load('y_train6000.npy')

In [3]:
X_train.shape

(50000, 3072)

In [19]:
reg = [0.001, 0.01, 0.1, 1.0, 10.0]
train_scores = []
val_scores = []
for reg in reg:
    qda = QuadraticDiscriminantAnalysis(reg_param=reg)
    qda.fit(X_train, y_train)
    train_score = qda.score(X_train, y_train)
    train_scores.append(train_score)
    val_score = qda.score(X_val, y_val)
    val_scores.append(val_score)
    print(f'When reg is {reg}, Train accuracy is {train_score}')
    print(f'When reg is {reg}, Validation accuracy is {val_score}')

C:\Users\Darren\Anaconda3\envs\MacbookPro\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


When reg is 0.001, Train accuracy is 0.0336
When reg is 0.001, Validation accuracy is 0.00625312656328164


C:\Users\Darren\Anaconda3\envs\MacbookPro\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


When reg is 0.01, Train accuracy is 0.02292
When reg is 0.01, Validation accuracy is 0.00591962647990662


C:\Users\Darren\Anaconda3\envs\MacbookPro\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


...

C:\Users\Darren\Anaconda3\envs\MacbookPro\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


When reg is 1.0, Train accuracy is 0.00102
When reg is 1.0, Validation accuracy is 0.007920626980156746


C:\Users\Darren\Anaconda3\envs\MacbookPro\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Darren\Anaconda3\envs\MacbookPro\lib\site-packages\sklearn\discriminant_analysis.py:715: RuntimeWarning: invalid value encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Darren\Anaconda3\envs\MacbookPro\lib\site-packages\sklearn\discriminant_analysis.py:718: RuntimeWarning: invalid value encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\Darren\Anaconda3\envs\MacbookPro\lib\site-packages\sklearn\discriminant_analysis.py:715: RuntimeWarning: invalid value encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))


When reg is 10.0, Train accuracy is 0.0043
When reg is 10.0, Validation accuracy is 0.003585125896281474


C:\Users\Darren\Anaconda3\envs\MacbookPro\lib\site-packages\sklearn\discriminant_analysis.py:718: RuntimeWarning: invalid value encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])


In [20]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)
train_score = qda.score(X_train, y_train)
train_scores.append(train_score)
val_score = qda.score(X_val, y_val)
val_scores.append(val_score)
print(f'When reg is {reg}, Train accuracy is {train_score}')
print(f'When reg is {reg}, Validation accuracy is {val_score}')

C:\Users\Darren\Anaconda3\envs\MacbookPro\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


When reg is 10.0, Train accuracy is 0.94172
When reg is 10.0, Validation accuracy is 0.0036685009171252294


In [13]:
pred_train = qda.predict(X_train)
pred_train_acc = np.mean(pred_train == y_train)
print(f'Train accuracy is {pred_train_acc}')

Train accuracy is 0.00102


In [7]:
train_df = pd.read_csv('./data/train_merge.csv')
val_df = pd.read_csv('./data/val_merge.csv')
train_dir = './data/train_set'
val_dir = './data/val_set'

In [8]:
X_val, y_val, label = datautils.gettestdata(val_df, val_dir, sample=len(val_df), class_mode='class')

Found 11994 validated image filenames belonging to 251 classes.


In [9]:
X_val.shape

(11994, 3072)

In [11]:
pred_val = qda.predict(X_val)
pred_val_acc = np.mean(pred_val == y_val)
print(f'Validation accuracy is {pred_val_acc}')

Validation accuracy is 0.0036685009171252294


In [16]:
train_score = qda.score(X_train, y_train)

In [17]:
train_score

0.00102

In [14]:
val_score = qda.score(X_val, y_val)

In [15]:
val_score

0.007920626980156746